# Exercise#3 Create a sequential chain

Use case:

User provides a *term*. The term is passed to an LLM to generate the *definition*. The definition is then passed to another LLM for scoring. The final output is the score.

Implementation:
1. Create a chain#1 for generating the definition
2. Create a chain#2 for scoring
3. Create a *sequential chain* with chain#1 and chain#2



In [1]:
from dotenv import load_dotenv
import os
import sys
import warnings
from IPython.display import JSON

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env1')

# Setting path so we can access the utils folder
sys.path.append('../')
sys.path.append('./')

#### Select LLM to use

**NOTE**

To try out different LLMs, call appropriate function below.

* You may see different or even incorrect results
* Adjust the prompts as needed




In [8]:
from utils.create_llm import (
                                create_google_llm, 
                                create_cohere_llm, 
                                create_gpt_llm, 
                                create_ai21_llm, 
                                create_anthropic_llm, 
                                create_hugging_face_llm
)

# Create the LLM for defining the term
define_term_llm = create_gpt_llm(api_key_prompt=True)

# Create the LLM for scoring the definition
definition_scoring_llm = create_ai21_llm(api_key_prompt=True)

Key:  OPENAI_API_KEY  already set in environment.
Key:  AI21_API_KEY  already set in environment.


## Chain#1 : define_term_chain

This chain takes a *term* as input and generates a definition for the given term.

In [9]:
from langchain.prompts import PromptTemplate

define_term_template = """
define the given term in under 25 words.

term: {term}

no need for preamble. Just provide the definition.
"""

In [10]:
from langchain.globals import set_debug
from langchain.globals import set_verbose

set_debug(False)
set_verbose(False)

define_term_prompt_template = PromptTemplate(
                        template=define_term_template,
                        input_variables = ["term"]
                    )


define_term_chain = define_term_prompt_template | define_term_llm

In [11]:
response=define_term_chain.invoke({"term": "energy"})
print(response)


The ability to do work or cause change, typically measured in joules, and can take various forms such as kinetic, potential, or thermal.


## Chain#2 : definition_scoring_chain

In [12]:
definition_scoring_template = """
You are given the definition of a term. Your job is to score the definition on a scale of 0 to 5.

definition: {definition}

no need to preamble. Just provide the score.
"""

definition_scoring_prompt_template = PromptTemplate(
                        template=definition_scoring_template,
                        input_variables = ["definition"]
                    )

definition_scoring_chain = definition_scoring_prompt_template | definition_scoring_llm

## Sequential chain

In [13]:
sequential_chain = {"definition": define_term_chain} | definition_scoring_chain

## Inference

In [14]:
response = sequential_chain.invoke({"term": "momentum"})

print(response)

Based on the definition you provided, I would score it as follows:

1. Clear and concise: The definition is clear and easy to understand, making it clear and concise.
2. Accurate: The definition accurately describes the concept of quantity of motion, along with its relation to mass and velocity.
3. Complete: The definition includes all the necessary components of quantity of motion, mass, and velocity.
4. Relevant: The definition is relevant to the study of physics and motion.
5. Well-explained: The definition provides a clear explanation of the concept of quantity of motion, along with its relationship with mass and velocity.

Final Score: 4.5
